In [ ]:
import os, sys
import time
import pickle
import gzip
from typing import *

import pyrealsense2 as rs
from matplotlib import pyplot as plt
import numpy as np
import cv2

def save_gzip(path, data):
    with gzip.open(path, 'wb') as f:
        pickle.dump(data, f)

def save_json(path, data):
    import json
    with open(path, "w") as f:
        json.dump(data, f)

In [ ]:
pipeline = rs.pipeline()
profile = pipeline.start()

In [ ]:
intr = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
# https://intelrealsense.github.io/librealsense/python_docs/_generated/pyrealsense2.intrinsics.html
camera_matrix = [[intr.fx, 0., intr.ppx],
                 [0., intr.fy, intr.ppy],
                 [0., 0., 1.]]
dist_coeff = intr.coeffs
calibrations = {
    'camera_matrix': camera_matrix,
    'dist_coeff': dist_coeff,
}

In [ ]:
# ---------------------------------------------------------------------------------------------------------------- #
# https://github.com/IntelRealSense/librealsense/blob/master/wrappers/python/examples/align-depth2color.py
# ---------------------------------------------------------------------------------------------------------------- #


# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)

In [ ]:
n_frame_per_save = 3

# Streaming loop
captures = []
try:
    for i in range(100000):
        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()
        # frames.get_depth_frame() is a 640x360 depth image

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        if (i+1) % n_frame_per_save == 0:
            captures.append((color_image.copy(),depth_image.copy()))

        # Remove background - Set pixels further than clipping_distance to grey
        grey_color = 153
        depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
        bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)

        # Render images:
        #   depth align to color on left
        #   depth on right
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((bg_removed, depth_colormap))

        cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Example', images)
        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()

In [ ]:
save_dir = 'dat/recorded/d415'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_gzip(os.path.join(save_dir, 'captures.gzip'), captures)
save_json(os.path.join(save_dir,'calibrations.json'), calibrations)